# Code to collect Rating Curve data
- Scraping of the rating curve of stream gauge data provided by USGS. 
#### Input:
Set of stream gages.
#### Output:
Rating curve data for every site. Formated where all data is stored in one data frame that can be accessed by inputting the specific gage.
#### Implementation:
Scraping is done by using beautifulsoup, requests staggered such to not overwhelem the system, data 

In [11]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

# Just the site number is needed
ca = pd.DataFrame(pd.read_csv('CA_valid_gages.csv')["Site No"])

In [12]:
ratings = pd.DataFrame()
noPageGage = pd.DataFrame()
noPageGage['Site No'] = []
for gageNum in ca['Site No']:
    try:
        url = ('https://nwis.waterdata.usgs.gov/nwisweb/data/ratings/exsa/USGS.' + str(gageNum) + '.exsa.rdb')
        response = requests.get(url)
        try:
            if response.status_code == 404:
                noPageGage.loc[len(noPageGage)] = {'Site No': str(gageNum)}
                continue;
        except:
            pass
        # Processing data from request
        soup = BeautifulSoup(response.text, 'html.parser')
        lines = soup.string.split('\n')
        lines = [line.strip() for line in lines if not line.strip().startswith('#')]
        data = [line.split("\t") for line in lines]
        df = pd.DataFrame(data[1:], columns=data[0])
        df['Site No'] = str(gageNum)
        col = df.pop('Site No')
        df.insert(0, 'Site No', col)
        ratings = pd.concat([ratings,df], ignore_index = True)
        time.sleep(3)
    except requests.exceptions.HTTPError as e:
        # Handle other HTTP errors here
        print(f"HTTP error {e.response.status_code} occurred: {url}")

In [ ]:
ratings.to_csv('rating_curve_wa.csv', index=False)

In [ ]:
ratings

,Site No,INDEP,SHIFT,DEP,STOR
0,10260500,16N,16N,16N,1S
1,10260500,0.79,-0.23,0.01,None
2,10260500,0.80,-0.23,0.02,None
3,10260500,0.81,-0.23,0.04,None
4,10260500,0.82,-0.23,0.05,None
...,...,...,...,...,...
162830,11525500,24.45,0.00,21033.14,None
162831,11525500,24.46,0.00,21065.17,None
162832,11525500,24.47,0.00,21097.21,None
162833,11525500,24.48,0.00,21129.28,*


In [ ]:
print(str(ratings[ratings["INDEP"].isna()].size) + '/' + str(ratings.size))
print(str(ratings[ratings["SHIFT"].isna()].size) + '/' + str(ratings.size))
print(str(ratings[ratings["DEP"].isna()].size) + '/' + str(ratings.size))
print(str(ratings[ratings["STOR"].isna()].size) + '/' + str(ratings.size))

0/814175
505/814175
505/814175
810000/814175


In [ ]:
ratings.describe()

,Site No,INDEP,SHIFT,DEP,STOR
count,162835,162835,162734,162734,835
unique,101,7917,787,147327,2
top,11467000,16N,0.00,16N,*
freq,6976,101,74968,101,734
